<img src='ipydata/표면처리.png' align='center' width="1500px"/>

<ul><li style="font-size:2em;">제조 데이터 준비</li></ul>
<div style="width:1200px; margin:0 auto;">
<a href="#"><img style="width:25%" src="ipydata/data.png"></a>
<a href="#"><img style="width:25%" src="ipydata/data2.png"></a>
</div>
<br>
<span style="font-size:20px;">사용대상: 표면처리 뿌리업종 내 아연도금 산출물에서 발생하는 이미지 제조데이터(Vision Machine 활용)</span>

<ul><li style="font-size:2em;">제조물</li></ul>
<div style="width:1200px; margin:0 auto;">
<a href="#"><img style="width:300px; height:200px;" src="ipydata/제품1.png"></a>
<a href="#"><img style="width:200px; height:200px;" src="ipydata/제품2.png"></a>
</div>
<br>
<span style="font-size:20px;">배관용 L자 연결 파이프(L자 관이음쇠) 아연도금</span>

<ul><li style="font-size:2em;">현장문제</li></ul>
<div style="width:1200px; margin:0 auto;">
<a href="#"><img style="width:400px; height:200px" src="ipydata/육안1.png"></a>
<a href="#"><img style="width:300px; height:200px" src="ipydata/육안2.png"></a>
</div>
<br>
<span style="font-size:20px;">공정시 산출물로 나오게 되는 L자 관이음쇠의 외관 품질 상태를 작업자가 하나하나 눈으로 체크하고 있으며, 이로 인한 노동력 낭비가 발생하고 있음</span>

<ul><li style="font-size:2em;">제조AI 개발 목적</li></ul>
<div style="width:1200px; margin:0 auto;">
<a href="#"><img style="width:400px; height:200px" src="ipydata/검사1.png"></a>
<a href="#"><img style="width:300px; height:200px" src="ipydata/검사2.png"></a>
</div>
<br>
<span style="font-size:20px;">비전 시스템을 활용하여 표면처리(아연도금) 산출물의 외관 품질 검사를 자동화하고, 분류할 수 있게 하기 위함</span>

# Library Install

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install sklearn
# !pip install seaborn
# !pip install tensorflow
# !pip install datetime
# !pip install matplotlib
# !pip install pydot
# !pip install graphviz

# Library Import

In [2]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import glob
from sklearn.utils import shuffle
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_squared_error
from math import sqrt

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Training Validation Tool Import
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve

In [ ]:
# Deep Learning Framework Import
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')  # Set GPU Use

In [ ]:
# GPU Use, not in condition -> Error
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True) # Set GPU Memory Useage Growth
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')   # Set Logical GPU
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")   # Print GPU Info
    except RuntimeError as e:
        print(e)

In [ ]:
# Image Preprocessing
def img_preprocess(img):
    img = Image.open(img)   # Open Image
    img = img.convert('RGB')   # Image Color Channel Change BGR->RGB
    img = img.resize((256,256))   # Image Resize for training
    img = np.asarray(img)/255   # Image Normalization
    return img  # return Image

In [ ]:
class AutoImageClassification:
    def __init__(self, path):
        self.train_df = self._create_df(path, '\학습')  # Training DataFrame Set
        self.test_df = self._create_df(path, '\테스트') # Test DataFrame Set
        
    @staticmethod
    def _create_df(path, path2):    # Make DataFrame for Training / Test
        files1 = glob.glob(path+path2+"\정상\*.png")    # Get Images from True Image Folder
        files2 = glob.glob(path+path2+"\불량\*.png")    # Get Images from False Image Folder
        df_n = pd.DataFrame()   # Initialize DataFrame for False
        df_p = pd.DataFrame()   # Initialize DataFrame for True
        df_n['name'] = [x for x in files2]  # False Image Name Insert into DataFrame's column name
        df_n['outcome'] = 0.      # False Image Value Insert into DataFrame's column outcome
        df_p['name'] = [x for x in files1]  # True Image Name Insert into DataFrame's column name
        df_p['outcome'] = 1.      # True Image Value Insert into DataFrame's column outcome
        df = pd.concat([df_n, df_p], axis = 0, ignore_index = True)  # Concat True / False Dataframe
        df = shuffle(df)        # Shuffle DataFame Rows
        return df
    
    def create_x_and_y(self):   # Get Data Pair (Image, Value)
        X = np.array([img_preprocess(p) for p in self.train_df.name.values])    # Get Image Data
        y = self.train_df.outcome.values    # Get Value Data from DataFrame
        return X, y

In [ ]:
aic = AutoImageClassification(path = os.getcwd())   # Initizlie AIC Class

In [ ]:
is_training = True # Set Training Trigger

In [ ]:
if is_training: # if Training Trigger True, Do underline
        X, y = aic.create_x_and_y()     # Get Training Datas

In [ ]:
def modelGen():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(input_shape=(256,256,3), filters=32, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(100))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
if is_training: # if Training Trigger True, Do Training
    model = modelGen()
    model.fit(x=X, y=y, epochs=3, validation_split=0.2, verbose=2)  # Model Training
    model.save('trainedModel.h5')    # Model Export
else:
    model = tf.keras.models.load_model("trainedModel.h5") # Get Trained Model

In [ ]:
model.summary() # Model Shape Showing

In [ ]:
plot_model(model)   # Model Architecture Showing

In [ ]:
x_test = np.array([img_preprocess(p) for p in aic.test_df.name.values]) # Get Test Images
y_test = aic.test_df.outcome.values # Get Test Values

In [ ]:
y_pred_ori = model.predict(x_test)  # Model Inference
y_pred = [round(y[0], 0) for y in y_pred_ori]   # Round Predict Values
print("accuracy  = ", accuracy_score(y_test, y_pred))   # Print Model Accuracy with Test Data
print("recall    = ", recall_score(y_test, y_pred))     # Print Model Recall with Test Data
print("precision = ", precision_score(y_test, y_pred))  # Print Model Score with Test Data
print("f1 score  = ", f1_score(y_test, y_pred))         # Print Model F1-Score with Test Data

In [ ]:
plt.hist(y_pred_ori)    # Predict Value Plot

In [ ]:
print('Keras Model Predict : ', y_pred) # Print Predict Values
rmse = sqrt(mean_squared_error(y_test, y_pred)) # Calculate RMSE Value with Predict Values and Real Values
print('Keras Model RMS : ', rmse)   # Print RMSE Value

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)    # Calculate ROC Curve Data
print(fpr, tpr, thresholds)                         # Print ROC Curve Data

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred, labels=[1, 0])   # Calculate Confusion Matrix Data
print(conf_matrix)                                              # Print Confusion Matrix Data

In [ ]:
recall = conf_matrix[0][0]/(conf_matrix[0][0]+conf_matrix[0][1])    # Calculate Recall Value from Confusion Matrix Data
fallout = conf_matrix[1][0]/(conf_matrix[1][0]+conf_matrix[1][1])   # Calculate Fallout Value from Confusion Matrix Data
plt.plot(fpr, tpr, 'o-', label="Logistic Regression")               # plot ROC Curve Data
plt.plot([0, 1], [0, 1], 'k--', label="random guess")
plt.plot([fallout], [recall], 'ro', ms=10)                          # plot Fallout Data
plt.xlabel('Fall-Out')                                              # Set x Label
plt.ylabel('Recall')                                                # Set y Label
plt.title('Receiver operating characteristic example')              # Set Title
plt.grid()                                                          # Set Grid
plt.legend()                                                        # Set Legend
plt.show()                                                          # Show Plot Image

In [ ]:
print(classification_report(y_test, y_pred, target_names=['class 0', 'class 1']))   # Print Report